# Figures for chapter data

## A DEM of the Zwalm catchment

In [ ]:
import openeo
import geopandas as gpd
import pandas as pd
import shapely
from pathlib import Path
import numpy as np
import pickle
import xarray as xr
import rioxarray
from rasterio import enums
import hvplot
import hvplot.xarray
import hvplot.pandas
import hvplot.dask
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import contextily as cx
from matplotlib import colors
from matplotlib.colors import LightSource
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
pad = Path(os.getcwd())
if pad.name == "Figures":
    pad_correct = pad.parent
    os.chdir(pad_correct)
from functions.plotting_functions import plot_discrete_raster, plot_continuous_raster
exec_download = False
write_fig = True

%load_ext autoreload
%autoreload 2

In [ ]:
#connection.list_collections()
connection.describe_collection('COPERNICUS_30')

More info found here: https://portal.opentopography.org/datasetMetadata?otCollectionID=OT.032021.4326.1

In [ ]:
collection = 'COPERNICUS_30'
shape_zwalm = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma.shp')
extent = shape_zwalm.total_bounds
print(extent)

In [ ]:
spatial_extent = {'west':extent[0],'east':extent[2],'south':extent[1],'north':extent[3]}
DEM = connection.load_collection(
    collection_id= collection,
    spatial_extent= spatial_extent
)
DEM = DEM.mask_polygon(shape_zwalm['geometry'].values[0])
if not os.path.exists('data/Zwalm_DEM'):
    os.makedirs('data/Zwalm_DEM')
if exec_download:
    DEM.download('data/Zwalm_DEM/DEM_Copernicus_30.nc',format = 'NetCDF')

In [ ]:
dem = xr.open_dataset('data/Zwalm_DEM/DEM_Copernicus_30.nc', decode_coords='all')
dem['DEM'].plot(cmap = 'terrain')

In [ ]:
dem.rio.crs


In [ ]:
dem_31370 = dem.rio.reproject(dst_crs='EPSG:31370', resampling = enums.Resampling.bilinear)
fig, ax = plt.subplots(constrained_layout = True)
dem_31370['DEM'].plot(ax = ax, cmap = 'Spectral_r', cbar_kwargs = {'label':'Height [m]'})
ax.set_title('')
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')

### Include Zwalm river on DEM

In [ ]:
shape_zwalm_31370 = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma_31370.shp')
VHA = gpd.read_file('data/VHA_waterlopen_20210204_GewVLA_Shapefile/Shapefile/Vhag.shp')

In [ ]:
VHA_zwalm= VHA[VHA['NAAM'] == 'Zwalmbeek']
VHA_zwalm

In [ ]:
VHA = VHA.set_crs(epsg=31370, allow_override=True)
VHA.crs

In [ ]:
shape_zwalm_31370.crs

In [ ]:
VHA.plot()

In [ ]:
shape_zwalm_31370['geometry']

In [ ]:
extent_31370 = shape_zwalm_31370.total_bounds
extent_31370

In [ ]:
VHA_zwalm_all = VHA[VHA['geometry'].within(shape_zwalm_31370['geometry'].values[0])]
#important to supply a raw geometry

In [ ]:
VHA_zwalm = pd.concat([VHA_zwalm, VHA_zwalm_all])

In [ ]:
VHA_zwalm.plot()

In [ ]:
dem_31370['DEM'].values[0].shape

In [ ]:
ls = LightSource(azdeg=315, altdeg=45)
plt.imshow(ls.hillshade(dem_31370['DEM'].values[0]))

In [ ]:
fig, ax = plt.subplots(constrained_layout = True)
dem_31370['DEM'].plot(ax = ax, cmap = 'Spectral_r', cbar_kwargs = {'label':'Height [m]'}, rasterized = True)
#ls = LightSource(azdeg=315, altdeg=45)
#ax.imshow(ls.hillshade(dem_31370['DEM'].values[0]),cmap = 'Spectral_r')#, cbar_kwargs = {'label':'Height [m]'})
VHA_zwalm.plot(ax = ax, color = 'orange')
ax.set_title('')
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
#cx.add_basemap(ax, crs = VHA_zwalm.crs, source = cx.providers.OpenStreetMap.Mapnik)
if not os.path.exists('Figures/Figures_chapter_data'):
    os.makedirs('Figures/Figures_chapter_data')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/DEM_zwalm.png', format = 'png')
    fig.savefig('Figures/Figures_chapter_data/DEM_zwalm.pdf', format = 'pdf')

In [ ]:
fig, ax = plt.subplots(constrained_layout = True)
fig, ax = plot_continuous_raster(
    fig, ax,dem_31370['DEM'].values[0],
    bounds = extent_31370,
    cmap = 'Spectral_r',
    hillshade=True,
    colorbar=True  
)

## Forcing data

### Rainfall

In [ ]:
# p_info = pd.read_pickle('data/Zwalm_data/pywaterinfo_output/P_info_dict.pickle')
# display(p_info)
# p_info['Elst']

In [ ]:
p_info = pd.read_pickle('data/Zwalm_data/preprocess_output/gdf_P_info.pkl')
hvplot.extension('bokeh')
p_info.hvplot(geo = True, crs = 31370, tiles = 'OSM')

https://scitools.org.uk/cartopy/docs/v0.15/examples/tube_stations.html

In [ ]:
# import cartopy.io.img_tiles as cimgt
# request = cimgt.OSM()
# #ax = plt.axes(projection = ccrs.epsg(31370))
# ax = plt.axes(projection = request.crs)
# #p_info.plot(ax = ax)
# ax.add_image(request, 8)
# plt.show()

In [ ]:
from cartopy.io.img_tiles import OSM
imagery = OSM()
fig = plt.figure(figsize = (14,6))
ax = plt.axes(projection = imagery.crs)
ax.add_image(imagery, 14)
p_info.plot(ax = ax, transform = ccrs.epsg(31370))
ax.set_title('Rainfall gauges')

https://geopandas.org/en/stable/gallery/plotting_basemap_background.html Zal stuk makkelijker zijn dan via cartopy zelf! 

https://contextily.readthedocs.io/en/latest/

In [ ]:
p_info

In [ ]:
fig, ax = plt.subplots(figsize = (5.7,5.5), constrained_layout = True)#figsize = (16,8))
p_info.plot(ax = ax,column = 'name',categorical = True, legend = True)
shape_zwalm_31370.plot(ax = ax, alpha = 0.5)
cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/rain_gauges.pdf', format = 'pdf')
#ax.set_title('Rain gauges')

In [ ]:
all_p_polygons = pickle.load(open('data/Zwalm_data/preprocess_output/all_p_polygon_combinations.pkl', "rb"))
length = len(all_p_polygons)
print(length)
fig, ax = plt.subplots(figsize = (5.5,5.5), constrained_layout = True)
all_p_polygons[length-1].plot(ax = ax, column = 'name', categorical = True)
p_info[0:-1].plot(ax = ax,column = 'name',categorical = True, legend = True, edgecolor = 'k')
cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/rain_thiessen_polygons.pdf', format = 'pdf')
#ax.set_titel('Thiessen Polygons

Attempt at combining the 2 above plots in one

In [ ]:
# fig, ax = plt.subplots(figsize = (5.5,5.5), constrained_layout = True)
# all_p_polygons[length-1].plot(ax = ax, column = 'name', categorical = True)
# p_info.plot(ax = ax,column = 'name',categorical = True, edgecolor = 'k', legend = True)
# #p_info.iloc[-1:].plot(ax = ax,categorical = True, edgecolor = 'k', color = 'red')
# cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
# #ax.legend([p_info['name'].to_list()[0:3],p_info['name'].to_list()[-1]])
# ax.set_aspect('equal','box')
# ax.set_xlabel('x [m]')
# ax.set_ylabel('y [m]')

https://matplotlib.org/stable/tutorials/colors/colors.html

In [ ]:
# fig, ax = plt.subplots(figsize = (5.5,5.5), constrained_layout = True)
# polygons_sorted = all_p_polygons[length-1].sort_values('name')
# p_info_sorted = p_info.sort_index()
# # polygons_sorted.plot(ax = ax, categorical = True, column = 'name')
# # p_info_sorted.plot(ax = ax, categorical = True, column = 'name')
# polygons_sorted.plot(ax = ax, color = ['red','green','blue'])
# p_info_sorted.plot(ax = ax, color = ['red','green','blue','purple'], edgecolor = 'k', legend = True)
# ax.legend()
# cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
# #ax.legend([p_info['name'].to_list()[0:3],p_info['name'].to_list()[-1]])
# ax.set_aspect('equal','box')
# ax.set_xlabel('x [m]')
# ax.set_ylabel('y [m]')

In [ ]:
cmap = plt.get_cmap('Set2')
cmap

In [ ]:
plt.style.use('default')
polygons_sorted = all_p_polygons[length-1].sort_values('name')
p_info_sorted = p_info.sort_index()
fig, ax = plt.subplots(figsize = (6,5.5), constrained_layout = True)
polygons_sorted.plot(ax = ax, color = [cmap.colors[0],cmap.colors[2],cmap.colors[-1]])#, color = cmap.colors[:3])#, color = ['#1f76b5','#d72628','#17bfce'])
p_info_sorted.plot(ax = ax,column = 'name',categorical = True, edgecolor = 'k', legend = True, cmap = 'Set2', legend_kwds = {'loc':'upper left'})
# p_info_sorted.plot(ax = ax, color = ['#1e76b4','#d72729','#e377c2','#16bece'], edgecolor = 'k',legend = True, column = 'name')
#p_info.iloc[-1:].plot(ax = ax,categorical = True, edgecolor = 'k', color = 'red')
cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
#ax.legend([p_info['name'].to_list()[0:3],p_info['name'].to_list()[-1]])
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
fig
if write_fig:
    plt.savefig('Figures/Figures_chapter_data/rain_combined.pdf',format = 'pdf')

TO DO: sorteer op manier zodat Elst, Maarke-Kerkem en Zingem eerst met DAN ronse er na! enkel op die manier kan zelfde kleur worden gegarandeerd! 

In [ ]:
polygons_sorted

In [ ]:
p_info_sorted

In [ ]:
# fig, ax = plt.subplots(figsize = (7,7))
# all_p_polygons[length-4].plot(ax = ax, column = 'name', categorical = True)
# p_info[0:-1].plot(ax = ax,column = 'name',categorical = True, legend = True, edgecolor = 'k')
# cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
# ax.set_aspect('equal','box')
# ax.set_xlabel('x [m]')
# ax.set_ylabel('y [m]')

https://www.statology.org/swap-columns-pandas/

In [ ]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
p_thiessen = pd.read_pickle('data/Zwalm_data/preprocess_output/zwalm_p_thiessen.pkl')
p_thiessen = p_thiessen.set_index('Timestamp')
p_thiessen = p_thiessen.drop('#_nonan_stations', axis = 'columns')
p_thiessen = swap_columns(p_thiessen,'Zingem','Ronse')
fig, axes = plt.subplots(2,1, constrained_layout = True)
p_thiessen.drop('P_thiessen', axis = 'columns').plot(
    ax =axes[0], cmap = 'Set2',alpha = 0.8#color = ['#1e76b4','#d72729','#e377c2','#16bece']
)
axes[0].set_ylabel('P [mm/h]')
axes[0].set_title('(a)')
axes[0].set_xlabel('Time')
p_thiessen['P_thiessen'].plot(ax = axes[1], color = cmap.colors[-2])
axes[1].set_ylabel('P [mm/h]')
axes[1].set_title('(b)')
axes[1].set_xlabel('Time')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/rain_timeseries.pdf', format = 'pdf')

### Evapotranspiration

In [ ]:
ep_info = pd.read_pickle('data/Zwalm_data/preprocess_output/gdf_EP_info.pkl')
fig, ax = plt.subplots(figsize = (5.5,5.5), constrained_layout = True)
ep_info.plot(ax = ax,column = 'name',categorical = True, legend = True)
shape_zwalm_31370.plot(ax = ax, alpha = 0.5)
cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/meteorological_stations.pdf', format = 'pdf')

In [ ]:
all_ep_polygons = pickle.load(open('data/Zwalm_data/preprocess_output/all_ep_polygon_combinations.pkl', "rb"))
length = len(all_ep_polygons)
print(length)
fig, ax = plt.subplots(figsize = (5.5,2.5), constrained_layout = True)
display(all_ep_polygons[length-1])
all_ep_polygons[length-1].plot(ax = ax, column = 'name', categorical = True)
ep_info[0:-1].plot(ax = ax,column = 'name',categorical = True, legend = True, edgecolor = 'k')
cx.add_basemap(ax, crs = ep_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/et_thiessen_polygons.pdf', format = 'pdf')

Again attempting to combine the 2 above plots!

In [ ]:
cmap_2 = plt.get_cmap('Accent')
cmap_2

In [ ]:
polygons_sorted = all_ep_polygons[length-1].sort_values('name')
ep_info_sorted = ep_info.sort_index()
fig, ax = plt.subplots(figsize = (5.5,5.5), constrained_layout = True)
polygons_sorted.plot(ax = ax, color = [cmap_2.colors[-4],cmap_2.colors[-1]])#, color = ['#8c564b','#16bece'])
ep_info_sorted.plot(ax = ax,column = 'name',categorical = True, edgecolor = 'k', legend = True, cmap = 'Accent')
#p_info.iloc[-1:].plot(ax = ax,categorical = True, edgecolor = 'k', color = 'red')
cx.add_basemap(ax, crs = p_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
#ax.legend([p_info['name'].to_list()[0:3],p_info['name'].to_list()[-1]])
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
if write_fig:
    plt.savefig('Figures/Figures_chapter_data/et_combined.pdf',format = 'pdf')

In [ ]:
display(polygons_sorted)
display(ep_info_sorted)

In [ ]:
# fig, ax = plt.subplots(figsize = (6.4,6.4), constrained_layout = True)
# all_ep_polygons[length-2].plot(ax = ax, column = 'name', categorical = True)
# ep_info.plot(ax = ax,column = 'name',categorical = True, legend = True, edgecolor = 'k')
# cx.add_basemap(ax, crs = ep_info.crs, source = cx.providers.OpenStreetMap.Mapnik)
# ax.set_aspect('equal','box')
# ax.set_xlabel('x [m]')
# ax.set_ylabel('y [m]')

In [ ]:
ep_thiessen = pd.read_pickle('data/Zwalm_data/preprocess_output/zwalm_ep_thiessen.pkl')
ep_thiessen = ep_thiessen.set_index('Timestamp')
ep_thiessen = ep_thiessen.drop('#_nonan_stations', axis = 'columns')
fig, axes = plt.subplots(2,1, constrained_layout = True)
ep_thiessen.drop(['EP_thiessen','EP_thiessen_ave_yearly'], axis = 'columns').plot(ax =axes[0], color = [cmap_2.colors[0],cmap_2.colors[-4],cmap_2.colors[-1]], alpha = 0.7)
axes[0].set_ylabel('PE [mm/h]')
axes[0].set_title('(a)')
axes[0].set_xlabel('Time')
ep_thiessen['EP_thiessen'].plot(ax = axes[1], c = cmap_2.colors[-2])
axes[1].set_ylabel('PE [mm/h]')
axes[1].set_title('(b)')
axes[1].set_xlabel('Time')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/et_timeseries.pdf', format = 'pdf')

In [ ]:
ep_thiessen.head()

## Flow Data

In [ ]:
Q_hour = pd.read_pickle('data/Zwalm_data/pywaterinfo_output/Q_hour.pkl')
Q_day = pd.read_pickle('data/Zwalm_data/pywaterinfo_output/Q_day.pkl')
Q_day = Q_day.set_index('Timestamp')
display(Q_hour.head())
display(Q_day.head())

In [ ]:
fig, ax = plt.subplots(figsize = (9,5))
Q_day['Value'].plot(ax = ax)
ax.set_ylabel(r'Q [m$^3$/s]')
ax.set_xlabel('Time')
if write_fig:
    plt.savefig('Figures/Figures_chapter_data/Q_daily.pdf',format = 'pdf')

## Landuse data

In [ ]:
landuse = rioxarray.open_rasterio('data/Zwalm_bodembedekking/wetransfer_landgebruik_2022-11-07_0921/'+
    'Landuse_Vlaanderen_Wallonie_final.sdat')
landuse = landuse.chunk('auto')#type:ignore
landuse_nonan = landuse.where(landuse != 255)

In [ ]:
landuse_nonan

In [ ]:
#landuse_nonan.isel(band = 0).hvplot(rasterize = True)

In [ ]:
landuse_zwalm = landuse_nonan.rio.clip(shape_zwalm_31370['geometry'].values,shape_zwalm_31370.crs)

In [ ]:
landuse_zwalm = landuse_zwalm.isel(band = 0).where(landuse_zwalm.isel(band = 0) != 255)
#landuse_zwalm

In [ ]:
np.sum(~np.isnan(landuse_zwalm.values))
landuse_zwalm

In [ ]:
#Taken from Fluves
colormap = colors.ListedColormap(
            [
                "#000000",
                "#387b00",
                "#64cf1b",
                 "#a47158",
                "#00bfff",
            ]
        )
labels = [
    'Urban',
    'Forest',
    'Pasture',
    'Agriculture',
    'Water'
]

In [ ]:
font_size = 13
plt.rcParams.update({'font.size': font_size})
fig, ax = plt.subplots(constrained_layout = True)
fig, ax = plot_discrete_raster(fig, ax, landuse_zwalm.values, bounds = extent_31370, labels = labels, cmap = colormap)
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/Landuse_zwalm.svg', format = 'svg')
    fig.savefig('Figures/Figures_chapter_data/Landuse_zwalm.pdf', format = 'pdf')

For egu poster: add the zwalm Rivers to the above plot

In [ ]:
ax.tick_params(axis='x', labelrotation=25)
VHA_zwalm.plot(ax = ax, color = "#00bfff", linewidth = 1.5)
if write_fig:
    fig.savefig('Figures/Figures_chapter_data/Landuse_zwalm_EGU.png', format = 'png',dpi = 500, transparent = True)
fig

In [ ]:
fig.get_size_inches()

Adapt the above figure to use in slides

In [ ]:
font_size = 13
plt.rcParams.update({'font.size': font_size})
#Taken from Fluves
colormap = colors.ListedColormap(
            [
                "#000000",
                "#387b00",
                "#64cf1b",
                 "#a47158",
                "#00bfff",
            ]
        )
labels = [
    'Urbaan',
    'Bos',
    'Weiland',
    'Landbouw',
    'Water'
]
fig, ax = plt.subplots(constrained_layout = True)
fig, ax = plot_discrete_raster(fig, ax, landuse_zwalm.values, bounds = extent_31370, labels = labels, cmap = colormap)
ax.set_aspect('equal','box')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.tick_params(axis='x', labelrotation=25)
pad = Path('Figures/presentation_12_04')
if not os.path.exists(pad):
    os.makedirs(pad)
if write_fig:
    fig.savefig(pad/'landuse.svg', format = 'svg')
fig

## LAI data

In [ ]:
pd_LAI_tseries = pd.read_pickle('data/LAI/LAI_timeseries.pkl')
pd_plotting = pd.read_pickle('data/LAI/LAI_plotting.pkl')

In [ ]:
fig, ax = plt.subplots(figsize = (9,6))
name_list_LAI = pd_plotting.columns.to_list()
print(name_list_LAI)
pd_LAI_tseries[name_list_LAI[1:-1]].plot(ax = ax, marker = '.', linestyle = 'None', cmap = colors.ListedColormap(["#387b00","#64cf1b","#a47158"]))
colors_used = [plt.gca().lines[i].get_color() for i in range(len(name_list_LAI)-2)]
pd_plotting[name_list_LAI[1:-1]].plot(ax = ax, color = colors_used)
og_names = ['Forest','Pasture','Agriculture']
interpol_names = ['Forest: interpolated','Pasture: interpolated',
'Agriculture: interpolated']
ax.legend(og_names + interpol_names, ncol = 2, loc = 'best')
ax.set_xlabel('Time')
ax.set_ylabel('LAI [-]')
if write_fig:
    plt.savefig('Figures/Figures_chapter_data/LAI_timeseries.pdf', format ='pdf')
fig

Repeat the above Figure for slides 12/04

In [ ]:
name_list_LAI

In [ ]:
font_size = 13
plt.rcParams.update({'font.size': font_size})
fig, ax = plt.subplots(constrained_layout = True)#figsize = (9,6))
pd_LAI_tseries[name_list_LAI[1:-1]].plot(ax = ax, marker = '.', linestyle = 'None', cmap = colors.ListedColormap(["#387b00","#64cf1b","#a47158"]))
colors_used = [plt.gca().lines[i].get_color() for i in range(len(name_list_LAI)-2)]
pd_plotting[name_list_LAI[1:-1]].plot(ax = ax, color = colors_used)
og_names = ['Bos','Weiland','Landouw']
interpol_names = ['Bos: geïnterpoleerd','Weiland: geïnterpoleerd',
'Landbouw: geïnterpoleerd']
ax.legend(og_names + interpol_names, ncol = 2, loc = 'best')
ax.set_xlabel('Tijd')
ax.set_ylabel('LAI [-]')
display(fig)
pad = Path('Figures/presentation_12_04')
if not os.path.exists(pad):
    os.makedirs(pad)
if write_fig:
    fig.savefig(pad/'LAI_timeseries.svg', format = 'svg')

In [ ]:

LAI_xr = xr.open_dataset('data/LAI/LAI_cube_Zwalm_landuse.nc')
fig, ax = plt.subplots(constrained_layout = True)
ax.set_aspect(1.5)
LAI_xr['LAI_pv'].isel(t=0).plot(cmap = 'Greens', cbar_kwargs = {'label':'LAI [-]'})
ax.set_xlabel('Longitude [°]')
ax.set_ylabel('Latitude [°]')
ax.tick_params(axis='x', labelrotation=25)
ax.set_title('')
if write_fig:
    fig.savefig(pad/'LAI_example.svg', format = 'svg')
import matplotlib
plt.rcParams.update(matplotlib.rcParamsDefault)
fig

In [ ]:
LAI_xr